In [1]:
import numpy as np
import scipy as sp
import scipy.optimize
import scipy.stats
import pandas as pd
import os
import pickle
import json
import matplotlib.pyplot as plt
import cvxpy as cp
import tqdm

from collections import defaultdict
from pathlib import Path

In [2]:
df = pd.read_pickle('../dataframe.pickle')

In [3]:
def find_peaks(data):
    peaks = np.zeros((2, 1))
    for i in range(1, int(data.size/2) - 1):
        if (data[1, i] > data[1, i-1] and data[1, i] > data[1, i+1]):
            peaks = np.append(peaks, [[data[0, i]], [data[1, i]]], axis=1)
    return peaks

def normalize_and_cutoff(data, order=np.inf, cutoff=0.05):
    data[1, :] /= np.linalg.norm(data[1, :], ord=order)
    idxs = np.squeeze(np.argwhere(data[1, :] > cutoff))
    return data[:, idxs]

def get_peaks_dict(datas, resolution=0.1):
    multiplier = 1 / resolution
    peaks_dict = defaultdict(int)
    for data in datas:
        _, npeaks = data.shape
        for i in range(npeaks):
            mlt_peak_val = int(round(data[0, i] * multiplier, 0))
            peaks_dict[mlt_peak_val] += 1
    return peaks_dict

def vectorize(datas, mapper, resolution):
    multiplier = 1/ resolution
    n = len(datas)
    d = len(mapper)
    samples = np.zeros((n, d))
    for i in range(n):
        data = datas[i]
        _, nd = data.shape
        for j in range(nd):
            peak = int(round(data[0, j] * multiplier, 0))
            if peak in mapper:
                samples[i, mapper[peak]] = max(data[1, j], samples[i, mapper[peak]])
                #samples[i, mapper[peak]] += data[1, j]
    return samples

def normalize_and_vectorize(
      data_list, 
      order=np.inf, 
      cutoff=0.05, 
      resolution=0.1, 
      min_peak_occur=2):
    peaks_data = [find_peaks(data) for data in data_list]
    normalized_data = [normalize_and_cutoff(d, order, cutoff) for d in peaks_data]
    peaks_dict = get_peaks_dict(normalized_data, resolution)
    peaks_mapper = dict()
    peaks = [key for key in peaks_dict.keys() if peaks_dict[key] >= min_peak_occur]
    for i in range(len(peaks)):
        peaks_mapper[peaks[i]] = i
    idx2peak = {v:k for k, v in peaks_mapper.items()}
    peak_seq = np.array([idx2peak[i] for i in range(len(idx2peak))]) * resolution
    vectorized_data = vectorize(normalized_data, peaks_mapper, resolution)
    return vectorized_data, peak_seq

In [4]:
cutoff=0.001
resolution=0.1
min_peak_occur=2
order=np.inf
vec_data, peak_seq = normalize_and_vectorize(
  df['mass_spec'], order, cutoff, resolution, min_peak_occur)
df['vec'] = list(vec_data)
npeaks = len(peak_seq)

In [5]:
idx = 0
for i, peak in enumerate(peak_seq):
  if abs(peak-151.0) < 1e-5:
    idx = i

In [6]:
def cvx_opt(A, Y, n=npeaks):
  x = cp.Variable((2, n))
  cost = cp.sum_squares(A @ x - Y)
  objective = cp.Minimize(cost)
  constraints = [x >= 0]
  prob = cp.Problem(objective, constraints)
  prob.solve(eps_rel=1e-12, eps_abs=1e-12)
  return x.value.T

In [9]:
def deconvolution(fld):
  loc = df['file'].str.find(f'Y{fld}_') != -1
  A = df.loc[loc, ['gfap_positive_count', 'gfap_negative_count']].values
  Y = np.array(df.loc[loc, ['vec']]['vec'].tolist())
  Y = Y / Y[:, idx].reshape((-1, 1))
  result = cvx_opt(A, Y)
  pos = result[:, 0]/result[:, 0].max()
  neg = result[:, 1]/result[:, 1].max()
  return pos, neg

positives = []
negatives = []
for i in tqdm.trange(1, 11):
  pos, neg = deconvolution(i)
  positives.append(pos)
  negatives.append(neg)
results = np.vstack([peak_seq] + positives + negatives).T

result_df = pd.DataFrame(results, 
    columns=(['m/z'] + 
             [f'gfap_positive_Y{i}' for i in range(1, 11)] + 
             [f'gfap_negative_Y{i}' for i in range(1, 11)]))
result_df.to_csv('cell_count_int_std.csv')

100%|██████████| 10/10 [20:00<00:00, 120.08s/it]


In [34]:
def deconvolution_all(df):
  A = df[['gfap_positive_count', 'gfap_negative_count']].values
  Y = np.array(df['vec'].tolist())
  Y = Y / Y[:, idx].reshape((-1, 1))
  result = cvx_opt(A, Y)
  pos = result[:, 0]/result[:, 0].max()
  neg = result[:, 1]/result[:, 1].max()
  return pos, neg

pos, neg = deconvolution_all(df)
results = np.vstack([peak_seq, pos, neg]).T

result_df = pd.DataFrame(results, 
    columns=('m/z', 'gfap_positive', 'gfap_negative'))
result_df.to_csv('cell_count_all_samples.csv')

In [6]:
df = pd.read_csv('cell_count_int_std.csv')
pos = [f'gfap_positive_Y{i + 1}' for i in range(10)]
neg = [f'gfap_negative_Y{i + 1}' for i in range(10)]
pos_samples = df[pos].values
neg_samples = df[neg].values
t_stat, p_val = sp.stats.ttest_ind(pos_samples, neg_samples, axis=1)
df['t-stat'] = t_stat
df['p-value'] = p_val
df.to_csv('cell_count_int_std_p_value.csv')

In [57]:
def clip(array, eps=1e-14):
  array[array < eps] = 0
  return array
df = pd.read_csv('cell_count_int_std.csv')
pos = [f'gfap_positive_Y{i + 1}' for i in range(10)]
neg = [f'gfap_negative_Y{i + 1}' for i in range(10)]
pos_samples = clip(df[pos].values)
neg_samples = clip(df[neg].values)

test_statistics, p_val = list(zip(*(
  sp.stats.ranksums(pos, neg) 
  for pos, neg in zip(pos_samples, neg_samples))))
df['ranksums_test_statistics'] = test_statistics
df['ranksums_p-value'] = p_val
df.to_csv('cell_count_int_std_ranksums.csv')